1. 게임 설정 입력 <br>
   ↓ <br> <br>
2. GameDifficultyCalculator 초기화 <br>
   ├── 모든 실력 수준 (0.1~0.9)에 대해 <br>
   ├── 모든 단계 (1~7)별로 <br>
   └── 기존 함수들로 성능 미리 계산 <br> 
   ↓ <br> <br>
3. 실제 데이터 입력 → UserPopulationEstimator <br>
   ├── 각 단계별 실패율 분석 <br>
   ├── 미리 계산된 프로필과 비교 <br>
   └── 유저 실력 분포 추정 <br>
   ↓ <br> <br>
4. AdaptiveMonteCarlo 시뮬레이션 <br>
   ├── 추정된 실력 분포로 유저 샘플링 <br>
   ├── 미리 계산된 성공률로 빠른 시뮬레이션 <br>
   └── 결과 분석 및 난이도 조정 권장 <br>

## 현재 게임 규칙

1단계2x2 / 총 그림 2개- 기본 카드: 2개- 트릭 카드: 0개제한 시간: 5초성공 리워드: 2원실패 리워드: 1원

2단계2x3 / 총 그림 3개- 기본 카드: 3개- 트릭 카드: 0개제한 시간: 7초성공 리워드: 2원

3단계3x4 / 총 그림 6개- 기본 카드: 6개- 트릭 카드: 0개제한 시간: 9초성공 리워드: 2원

4단계4x4 / 총 그림 8개- 기본 카드: 8개- 트릭 카드: 0개제한 시간: 13초성공 리워드: 2원

5단계4x4 / 총 그림 8개- 기본 카드: 6개- 트릭 카드: 2개제한 시간: 15초성공 리워드: 3원

6단계4x5 / 총 그림 10개- 기본 카드: 10개- 트릭 카드: 0개제한 시간: 21초성공 리워드: 3원

7단계4x5 / 총 그림 10개- 기본 카드: 8개- 트릭 카드: 2개제한 시간: 25초성공 리워드: 6원

In [ ]:
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
import numpy as np
import pandas as pd
import scipy.sparse
import scipy.sparse.linalg
import sys
# 맥과 윈도우 os 판별하고, 시각화에서 한글 꺠지지 않도록 설정하는 코드
if sys.platform == 'darwin':
    plt.rcParams['font.family'] = 'AppleGothic'
else:
    plt.rcParams['font.family'] = 'Malgun Gothic'

import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import scipy.sparse
import scipy.sparse.linalg
from typing import Tuple, Dict, Any, Optional

def remove_last_row_column(sparse_matrix):
    """행렬에서 마지막 행과 열을 제거하는 함수"""
    num_states = sparse_matrix.shape[0]
    row_mask = np.ones(sparse_matrix.shape[0], dtype=bool)
    row_mask[num_states-1] = False
    col_mask = np.ones(sparse_matrix.shape[1], dtype=bool)
    col_mask[num_states-1] = False
    return sparse_matrix[row_mask][:,col_mask]

def _build_matrix_from_adjacency(adjacency_list: Dict, entire_matrix: bool, extra_info: bool):
    """card_matching.ipynb와 동일한 행렬 구성"""
    num_states = len(adjacency_list)
    markov_chain = scipy.sparse.lil_matrix((num_states, num_states))

    # Build enumeration for states - card_matching.ipynb와 동일
    sorted_keys = sorted(list(adjacency_list.keys()), key=lambda pair: -sum(pair))
    keys_to_index = {key: index for index, key in enumerate(sorted_keys)}

    for state, neighbors in adjacency_list.items():
        adjacency_list[state] = {k: v for k, v in neighbors.items() if v > 1e-10}
        for destination, probability in neighbors.items():
            markov_chain[(keys_to_index[state], keys_to_index[destination])] = probability
    
    if not entire_matrix:  # Get block matrix for fundamental matrix calculation
        to_return = remove_last_row_column(markov_chain)
    else:
        to_return = scipy.sparse.csr_matrix(markov_chain)

    if extra_info:
        return to_return, {"adjacency_list": adjacency_list, "keys_to_index": keys_to_index, "sorted_keys": sorted_keys}
    return to_return

def get_optimal_baseline_markov_chain(number_of_pairs: int, entire_matrix: bool = False, extra_info: bool = False):
    """card_matching.ipynb와 동일한 완벽한 기억력 베이스라인"""
    queue = [(0, 2 * number_of_pairs)]
    adjacency_list = {}

    while queue:
        curr_state = queue.pop()
        alpha, beta = curr_state
    
        if alpha < 0 or beta < 0:
            continue
  
        next_states = {}
        # card_matching.ipynb의 정확한 로직
        if alpha > beta or (alpha + beta) % 2 == 1:  # Know more knowns than unknowns or there is a duplicate in the flipped cards
            next_states[(alpha - 1, beta)] = 1  # 완전 기억 케이스
        elif alpha == 0 and beta == 0:
            next_states[(0, 0)] = 1
        elif alpha == beta:
            next_states[(max(0, alpha - 1), beta - 1)] = 1
        else: 
            # 완전 기억 케이스 - card_matching.ipynb와 동일
            next_states[(alpha - 1, beta - 1)] = alpha / beta 
            next_states[(alpha + 2, beta - 2)] = (1 - alpha/beta) * (1 - (alpha+1)/(beta-1))
            next_states[(alpha + 1, beta - 2)] = (1 - alpha/beta) * (alpha / (beta - 1))
            next_states[(alpha, beta - 2)] = (1 - alpha/beta) * 1 / (beta - 1)

        # exclude calculating transitions for states with 0 probability to reach
        queue.extend(state for state, probability in next_states.items() 
                    if state not in adjacency_list and probability > 0)
        adjacency_list[curr_state] = next_states

    # Clear out not possible states
    for state, neighbors in adjacency_list.items():
        adjacency_list[state] = {k: v for k, v in neighbors.items() if v > 0}

    return _build_matrix_from_adjacency(adjacency_list, entire_matrix, extra_info)

def get_imperfect_memory_markov_chain(number_of_pairs: int, memory_decay_factor: float = 0.1, 
                                     entire_matrix: bool = False, extra_info: bool = False):
    """기억력 저하로 인한 성능 감소 모델 - 턴 수가 증가하도록 설계"""
    queue = [(0, 2 * number_of_pairs)]
    adjacency_list = {}

    while queue:
        curr_state = queue.pop()
        alpha, beta = curr_state
    
        if alpha < 0 or beta < 0:
            continue
  
        next_states = {}
        
        # 게임 종료 조건들
        if alpha == 0 and beta == 0:
            next_states[(0, 0)] = 1
        elif beta == 0:
            next_states[(0, 0)] = 1
        elif beta == 1:
            next_states[(0, 0)] = 1
        elif (alpha + beta) % 2 == 1:
            next_states[(max(0, alpha - 1), beta)] = 1
        else:
            # 기억력 저하 효과 적용
            base_success_prob = alpha / beta if alpha <= beta else 0.9
            
            # memory_decay_factor가 클수록 성능 저하 (더 많은 턴 필요)
            degraded_success_prob = base_success_prob * (1 - memory_decay_factor)
            
            # 매칭 성공 (기억한 정보로 올바른 선택)
            success_prob = min(degraded_success_prob, 0.95)  # 최대 95%로 제한
            next_states[(max(0, alpha - 1), beta - 2)] = success_prob
            
            # 실패 케이스들 (기억력 저하로 인한 비효율적 선택)
            fail_prob = 1 - success_prob
            
            if fail_prob > 0:
                # 완전 실패: 새로운 정보만 얻고 매칭 실패
                next_states[(min(alpha + 2, beta), beta - 2)] = fail_prob * 0.6
                
                # 부분 실패: 일부 정보만 기억하고 매칭 실패  
                next_states[(min(alpha + 1, beta), beta - 2)] = fail_prob * 0.3
                
                # 정보 손실: 기존 정보도 일부 잃음
                next_states[(max(0, alpha - 1), beta)] = fail_prob * 0.1

        queue.extend(state for state, probability in next_states.items() 
                    if state not in adjacency_list and probability > 0)
        adjacency_list[curr_state] = next_states

    # 0 확률 상태 제거
    for state, neighbors in adjacency_list.items():
        adjacency_list[state] = {k: v for k, v in neighbors.items() if v > 0}

    return _build_matrix_from_adjacency(adjacency_list, entire_matrix, extra_info)

def get_case_statistics(markov_matrix, n_pairs: int) -> np.ndarray:
    """card_matching.ipynb와 동일한 분포 계산"""
    starting_point = np.zeros(markov_matrix.shape[0])
    starting_point[0] = 1

    first_chance_to_win = markov_matrix.T ** n_pairs
    prob_vector = first_chance_to_win @ starting_point
    chance_to_finish_with_failures = [prob_vector[-1]]

    # card_matching.ipynb와 동일한 범위
    for number_of_failures in range(n_pairs):
        next_prob_vector = markov_matrix.T @ prob_vector
        chance_to_finish_with_failures.append(next_prob_vector[-1] - prob_vector[-1])
        prob_vector = next_prob_vector
    
    return np.array(chance_to_finish_with_failures)

def calculate_turn_penalty(skill_level: float, baseline_turns: float) -> float:
    """실력 수준에 따른 턴 수 증가 계산"""
    
    # skill_level: 1.0 (완벽) -> 0.0 (최악)
    # 베이스라인 대비 턴 수 증가율
    
    if skill_level >= 0.9:
        penalty_multiplier = 1.0  # 베이스라인 수준
    elif skill_level >= 0.7:
        penalty_multiplier = 1.3  # 30% 증가
    elif skill_level >= 0.5:
        penalty_multiplier = 1.8  # 80% 증가
    elif skill_level >= 0.3:
        penalty_multiplier = 2.5  # 150% 증가
    elif skill_level >= 0.1:
        penalty_multiplier = 3.5  # 250% 증가
    else:
        penalty_multiplier = 5.0  # 400% 증가
    
    return baseline_turns * penalty_multiplier

def skill_level_to_memory_decay(skill_level: float) -> float:
    """실력 수준을 memory_decay_factor로 변환"""
    # 높은 실력 = 낮은 decay, 낮은 실력 = 높은 decay
    return max(0.0, min(0.8, (1.0 - skill_level) * 0.8))

def get_user_performance_metrics(n_pairs: int, skill_level: float) -> Dict[str, float]:
    """
    실력 수준에 따른 성과 지표 계산 - 베이스라인 대비 턴 수 증가 중심
    """
    # 베이스라인 계산
    baseline_matrix = get_optimal_baseline_markov_chain(n_pairs, entire_matrix=True)
    baseline_stats = get_case_statistics(baseline_matrix, n_pairs)
    baseline_expected_failures = sum(i * prob for i, prob in enumerate(baseline_stats))
    baseline_turns = baseline_expected_failures + n_pairs
    baseline_completion_rate = sum(baseline_stats)
    
    # 사용자 성능 계산
    if skill_level >= 0.95:
        # 거의 완벽한 사용자는 베이스라인과 동일
        user_stats = baseline_stats
        user_expected_failures = baseline_expected_failures
        user_completion_rate = baseline_completion_rate
    else:
        # 기억력 저하가 있는 사용자
        memory_decay = skill_level_to_memory_decay(skill_level)
        user_matrix = get_imperfect_memory_markov_chain(n_pairs, memory_decay, entire_matrix=True)
        user_stats = get_case_statistics(user_matrix, n_pairs)
        user_completion_rate = sum(user_stats)
        
        if user_completion_rate > 0:
            user_expected_failures = sum(i * prob for i, prob in enumerate(user_stats)) / user_completion_rate
        else:
            user_expected_failures = float('inf')
    
    # 실제 턴 수 (패널티 적용)
    actual_turns = calculate_turn_penalty(skill_level, baseline_turns)
    
    # 성능 지표
    turn_increase_ratio = actual_turns / baseline_turns if baseline_turns > 0 else float('inf')
    efficiency_score = baseline_turns / actual_turns if actual_turns > 0 else 0.0
    
    return {
        'skill_level': skill_level,
        'baseline_turns': baseline_turns,
        'actual_turns': actual_turns,
        'turn_increase_ratio': turn_increase_ratio,
        'efficiency_score': efficiency_score,
        'expected_failures': user_expected_failures,
        'completion_rate': user_completion_rate,
        'baseline_completion_rate': baseline_completion_rate
    }

# def monte_carlo_user_simulation(n_pairs: int, user_skill_distribution: list, 
#                                skill_levels: list = [0.1, 0.3, 0.5, 0.7, 0.9],
#                                num_simulations: int = 10000) -> dict:
#     """유저 실력 분포를 기반으로 한 몬테카를로 시뮬레이션"""
    
#     # 베이스라인 계산
#     baseline_matrix = get_optimal_baseline_markov_chain(n_pairs, entire_matrix=True)
#     baseline_stats = get_case_statistics(baseline_matrix, n_pairs)
#     baseline_expected_failures = sum(i * prob for i, prob in enumerate(baseline_stats))
#     baseline_turns = baseline_expected_failures + n_pairs
    
#     results = []
    
#     for simulation in range(num_simulations):
#         # 유저 실력 샘플링
#         user_skill = np.random.choice(skill_levels, p=user_skill_distribution)
        
#         # 해당 실력에 따른 실제 턴 수 계산
#         actual_turns = calculate_turn_penalty(user_skill, baseline_turns)
        
#         # 게임 경제성 계산
#         ad_revenue_per_turn = 0.05  # 턴당 광고 수익
#         completion_reward = 3.0
        
#         ad_revenue = actual_turns * ad_revenue_per_turn
#         net_profit = ad_revenue - completion_reward
        
#         results.append({
#             'user_skill': user_skill,
#             'baseline_turns': baseline_turns,
#             'actual_turns': actual_turns,
#             'turn_increase': actual_turns - baseline_turns,
#             'turn_increase_ratio': actual_turns / baseline_turns,
#             'ad_revenue': ad_revenue,
#             'net_profit': net_profit
#         })
    
#     # 통계 계산
#     avg_turns = np.mean([r['actual_turns'] for r in results])
#     avg_turn_increase = np.mean([r['turn_increase'] for r in results])
#     avg_profit = np.mean([r['net_profit'] for r in results])
    
#     return {
#         'simulations': results,
#         'baseline_turns': baseline_turns,
#         'avg_actual_turns': avg_turns,
#         'avg_turn_increase': avg_turn_increase,
#         'avg_turn_increase_ratio': avg_turns / baseline_turns,
#         'avg_profit': avg_profit,
#         'total_simulations': num_simulations
#     }



In [ ]:
def example_usage():
    """사용 예시"""
    n_pairs = 7
    
    print("=== 베이스라인 vs 다양한 실력 수준 비교 ===")
    skill_levels = [1.0, 0.8, 0.6, 0.4, 0.2, 0.0]
    
    for skill in skill_levels:
        metrics = get_user_performance_metrics(n_pairs, skill)
        print(f"실력 {skill:.1f}: 베이스라인 {metrics['baseline_turns']:.2f}턴 → "
              f"실제 {metrics['actual_turns']:.2f}턴 "
              f"(+{metrics['turn_increase_ratio']:.1%})")
    
    print("\n=== 몬테카를로 시뮬레이션 ===")
    # 실제적인 유저 분포 (초보자가 많음)
    user_distribution = [0.4, 0.3, 0.2, 0.08, 0.02]  # [0.1, 0.3, 0.5, 0.7, 0.9]
    
    results = monte_carlo_user_simulation(
        n_pairs=n_pairs,
        user_skill_distribution=user_distribution,
        num_simulations=10000
    )
    
    print(f"베이스라인 평균 턴 수: {results['baseline_turns']:.2f}")
    print(f"실제 평균 턴 수: {results['avg_actual_turns']:.2f}")
    print(f"평균 턴 수 증가: +{results['avg_turn_increase']:.2f}턴 "
          f"(+{(results['avg_turn_increase_ratio']-1)*100:.1f}%)")
    print(f"평균 순이익: ${results['avg_profit']:.3f}")
    
    return results

if __name__ == "__main__":
    example_usage()

=== 베이스라인 vs 다양한 실력 수준 비교 ===
실력 1.0: 베이스라인 10.78턴 → 실제 10.78턴 (+100.0%)
실력 0.8: 베이스라인 10.78턴 → 실제 14.01턴 (+130.0%)
실력 0.6: 베이스라인 10.78턴 → 실제 19.40턴 (+180.0%)
실력 0.4: 베이스라인 10.78턴 → 실제 26.95턴 (+250.0%)
실력 0.2: 베이스라인 10.78턴 → 실제 37.73턴 (+350.0%)
실력 0.0: 베이스라인 10.78턴 → 실제 53.90턴 (+500.0%)

=== 몬테카를로 시뮬레이션 ===
베이스라인 평균 턴 수: 10.78
실제 평균 턴 수: 28.45
평균 턴 수 증가: +17.67턴 (+164.0%)
평균 순이익: $-1.577


In [ ]:
# 

In [ ]:
def validate_model_7pairs():
    """7페어 게임에서 모델 검증 - 각 실력별 턴 분포 상세 분석"""
    n_pairs = 7
    skill_levels = [1.0, 0.8, 0.6, 0.4, 0.2, 0.1]
    
    print("=== 7페어 게임 모델 검증 ===\n")
    
    # 베이스라인 계산
    baseline_matrix = get_optimal_baseline_markov_chain(n_pairs, entire_matrix=True)
    baseline_stats = get_case_statistics(baseline_matrix, n_pairs)
    baseline_expected_failures = sum(i * prob for i, prob in enumerate(baseline_stats))
    baseline_turns = baseline_expected_failures + n_pairs
    baseline_completion_rate = sum(baseline_stats)
    
    print(f"🎯 베이스라인 (완벽한 기억력):")
    print(f"   예상 턴 수: {baseline_turns:.2f}")
    print(f"   완료율: {baseline_completion_rate:.1%}")
    print(f"   분포 (처음 10개): {baseline_stats[:10]}")
    print()
    
    results = {}
    
    for skill in skill_levels:
        print(f"🧠 실력 수준: {skill:.1f}")
        print("-" * 50)
        
        # 기본 성능 지표
        metrics = get_user_performance_metrics(n_pairs, skill)
        
        # 상세 분포 계산
        if skill >= 0.95:
            user_stats = baseline_stats
            user_completion_rate = baseline_completion_rate
        else:
            memory_decay = skill_level_to_memory_decay(skill)
            user_matrix = get_imperfect_memory_markov_chain(n_pairs, memory_decay, entire_matrix=True)
            user_stats = get_case_statistics(user_matrix, n_pairs)
            user_completion_rate = sum(user_stats)
        
        # 턴별 확률 분포 (7턴부터 시작)
        turns_range = range(7, min(7 + len(user_stats), 25))
        turn_distribution = {}
        
        for i, turn in enumerate(turns_range):
            if i < len(user_stats):
                turn_distribution[turn] = user_stats[i]
            else:
                turn_distribution[turn] = 0.0
        
        # 조건부 기댓값 (완료한 경우만)
        if user_completion_rate > 0:
            conditional_expected_failures = sum(i * prob for i, prob in enumerate(user_stats)) / user_completion_rate
            conditional_expected_turns = conditional_expected_failures + n_pairs
        else:
            conditional_expected_failures = float('inf')
            conditional_expected_turns = float('inf')
        
        # 결과 저장
        results[skill] = {
            'baseline_turns': metrics['baseline_turns'],
            'penalty_turns': metrics['actual_turns'],
            'conditional_turns': conditional_expected_turns,
            'completion_rate': user_completion_rate,
            'distribution': user_stats,
            'turn_distribution': turn_distribution,
            'memory_decay': skill_level_to_memory_decay(skill) if skill < 0.95 else 0.0
        }
        
        print(f"   메모리 감쇠: {results[skill]['memory_decay']:.3f}")
        print(f"   조건부 기댓값: {conditional_expected_turns:.2f}턴")
        print(f"   패널티 적용 후: {metrics['actual_turns']:.2f}턴")
        print(f"   완료율: {user_completion_rate:.1%}")
        print(f"   증가율: +{((metrics['actual_turns']/metrics['baseline_turns'])-1)*100:.1f}%")
        
        # 주요 턴별 확률 표시
        print(f"   주요 턴 확률:")
        for turn in [7, 8, 9, 10, 12, 15, 20]:
            if turn in turn_distribution:
                prob = turn_distribution[turn]
                if prob > 0.01:  # 1% 이상인 경우만 표시
                    print(f"     {turn}턴: {prob:.1%}")
        
        print()
    
    # 요약 표
    print("\n📊 요약 표:")
    print("실력 | 베이스라인 | 조건부 | 패널티후 | 완료율 | 증가율")
    print("-----|----------|--------|----------|--------|--------")
    
    for skill in skill_levels:
        r = results[skill]
        increase_pct = ((r['penalty_turns']/r['baseline_turns'])-1)*100
        print(f"{skill:4.1f} | {r['baseline_turns']:8.2f} | {r['conditional_turns']:6.2f} | "
              f"{r['penalty_turns']:8.2f} | {r['completion_rate']:6.1%} | +{increase_pct:5.1f}%")
    
    return results

# 실행
validation_results = validate_model_7pairs()

=== 7페어 게임 모델 검증 ===

🎯 베이스라인 (완벽한 기억력):
   예상 턴 수: 10.78
   완료율: 100.0%
   분포 (처음 10개): [7.40000740e-06 8.28800829e-04 2.79720280e-02 2.83449883e-01
 5.67521368e-01 1.19288119e-01 9.32400932e-04 0.00000000e+00]

🧠 실력 수준: 1.0
--------------------------------------------------
   메모리 감쇠: 0.000
   조건부 기댓값: 10.78턴
   패널티 적용 후: 10.78턴
   완료율: 100.0%
   증가율: +0.0%
   주요 턴 확률:
     9턴: 2.8%
     10턴: 28.3%
     12턴: 11.9%

🧠 실력 수준: 0.8
--------------------------------------------------
   메모리 감쇠: 0.160
   조건부 기댓값: 11.77턴
   패널티 적용 후: 14.01턴
   완료율: 81.6%
   증가율: +30.0%
   주요 턴 확률:
     10턴: 8.0%
     12턴: 23.3%

🧠 실력 수준: 0.6
--------------------------------------------------
   메모리 감쇠: 0.320
   조건부 기댓값: 11.79턴
   패널티 적용 후: 19.40턴
   완료율: 67.1%
   증가율: +80.0%
   주요 턴 확률:
     10턴: 7.4%
     12턴: 18.7%

🧠 실력 수준: 0.4
--------------------------------------------------
   메모리 감쇠: 0.480
   조건부 기댓값: 11.82턴
   패널티 적용 후: 26.95턴
   완료율: 51.9%
   증가율: +150.0%
   주요 턴 확률:
     9턴: 1.2%
     10턴: 6.2%
  

In [ ]:
# v5에서 베이스라인 확인
def compare_with_original():
    """card_matching.ipynb와 v5 베이스라인 비교"""
    n_pairs = 7
    
    # v5 베이스라인
    baseline_matrix = get_optimal_baseline_markov_chain(n_pairs, entire_matrix=True)
    baseline_stats = get_case_statistics(baseline_matrix, n_pairs)
    baseline_expected_failures = sum(i * prob for i, prob in enumerate(baseline_stats))
    baseline_turns = baseline_expected_failures + n_pairs
    
    print("=== v5 베이스라인 vs card_matching.ipynb 비교 ===")
    print(f"v5 베이스라인 턴 수: {baseline_turns:.2f}")
    print(f"v5 베이스라인 분포: {baseline_stats[:8]}")
    print()
    
    # card_matching.ipynb 원본 결과
    original_result = np.array([7.40000740e-06, 8.28800829e-04, 2.79720280e-02, 2.83449883e-01,
                               5.67521368e-01, 1.19288119e-01, 9.32400932e-04, 0.00000000e+00])
    original_expected_failures = sum(i * prob for i, prob in enumerate(original_result))
    original_turns = original_expected_failures + n_pairs
    
    print(f"card_matching.ipynb 턴 수: {original_turns:.2f}")
    print(f"card_matching.ipynb 분포: {original_result}")
    print()
    
    print("11턴 (4실패) 확률 비교:")
    print(f"card_matching.ipynb: {original_result[4]:.1%}")
    if len(baseline_stats) > 4:
        print(f"v5 베이스라인: {baseline_stats[4]:.1%}")
    else:
        print("v5 베이스라인: 해당 인덱스 없음")
    
    return {
        'v5_baseline': baseline_stats,
        'original': original_result,
        'v5_turns': baseline_turns,
        'original_turns': original_turns
    }

compare_with_original()


=== v5 베이스라인 vs card_matching.ipynb 비교 ===
v5 베이스라인 턴 수: 10.78
v5 베이스라인 분포: [7.40000740e-06 8.28800829e-04 2.79720280e-02 2.83449883e-01
 5.67521368e-01 1.19288119e-01 9.32400932e-04 0.00000000e+00]

card_matching.ipynb 턴 수: 10.78
card_matching.ipynb 분포: [7.40000740e-06 8.28800829e-04 2.79720280e-02 2.83449883e-01
 5.67521368e-01 1.19288119e-01 9.32400932e-04 0.00000000e+00]

11턴 (4실패) 확률 비교:
card_matching.ipynb: 56.8%
v5 베이스라인: 56.8%


{'v5_baseline': array([7.40000740e-06, 8.28800829e-04, 2.79720280e-02, 2.83449883e-01,
        5.67521368e-01, 1.19288119e-01, 9.32400932e-04, 0.00000000e+00]),
 'original': array([7.40000740e-06, 8.28800829e-04, 2.79720280e-02, 2.83449883e-01,
        5.67521368e-01, 1.19288119e-01, 9.32400932e-04, 0.00000000e+00]),
 'v5_turns': 10.77924297924298,
 'original_turns': 10.779242978421}

In [ ]:
def verify_baseline_match():
    """베이스라인이 card_matching.ipynb와 일치하는지 확인"""
    n_pairs = 7
    
    # v5 베이스라인
    baseline_matrix = get_optimal_baseline_markov_chain(n_pairs, entire_matrix=True)
    baseline_stats = get_case_statistics(baseline_matrix, n_pairs)
    baseline_expected_failures = sum(i * prob for i, prob in enumerate(baseline_stats))
    baseline_turns = baseline_expected_failures + n_pairs
    
    # card_matching.ipynb 원본 결과
    original_result = np.array([7.40000740e-06, 8.28800829e-04, 2.79720280e-02, 2.83449883e-01,
                               5.67521368e-01, 1.19288119e-01, 9.32400932e-04, 0.00000000e+00])
    original_expected_failures = sum(i * prob for i, prob in enumerate(original_result))
    original_turns = original_expected_failures + n_pairs
    
    print("=== 베이스라인 검증 ===")
    print(f"card_matching.ipynb 원본:")
    print(f"  턴 수: {original_turns:.2f}")
    print(f"  11턴 확률: {original_result[4]:.1%}")
    print(f"  분포: {original_result}")
    print()
    
    print(f"v5 수정된 베이스라인:")
    print(f"  턴 수: {baseline_turns:.2f}")
    if len(baseline_stats) > 4:
        print(f"  11턴 확률: {baseline_stats[4]:.1%}")
    print(f"  분포: {baseline_stats}")
    print()
    
    # 차이 확인
    diff = abs(baseline_turns - original_turns)
    print(f"턴 수 차이: {diff:.4f}")
    
    if diff < 0.01:
        print("✅ 베이스라인 일치!")
    else:
        print("❌ 베이스라인 불일치")
    
    return baseline_stats, original_result

# 검증 실행
baseline_verification = verify_baseline_match()

=== 베이스라인 검증 ===
card_matching.ipynb 원본:
  턴 수: 10.78
  11턴 확률: 56.8%
  분포: [7.40000740e-06 8.28800829e-04 2.79720280e-02 2.83449883e-01
 5.67521368e-01 1.19288119e-01 9.32400932e-04 0.00000000e+00]

v5 수정된 베이스라인:
  턴 수: 10.78
  11턴 확률: 56.8%
  분포: [7.40000740e-06 8.28800829e-04 2.79720280e-02 2.83449883e-01
 5.67521368e-01 1.19288119e-01 9.32400932e-04 0.00000000e+00]

턴 수 차이: 0.0000
✅ 베이스라인 일치!


### 시뮬



In [ ]:
# 게임 변수
MAX_RETRIES = 3      # 최대 재시도 횟수
AD_REVENUE = 1.0     # 광고 수익

# 게임 단계 설정
GAME_STAGES = [
    {'pairs': 2, 'time_limit': 5.0, 'reward': 2},    # 1단계: 2x2, 2페어
    {'pairs': 3, 'time_limit': 7.0, 'reward': 2},    # 2단계: 2x3, 3페어  
    {'pairs': 6, 'time_limit': 9.0, 'reward': 2},    # 3단계: 3x4, 6페어
    {'pairs': 8, 'time_limit': 13.0, 'reward': 2},   # 4단계: 4x4, 8페어
    {'pairs': 8, 'time_limit': 15.0, 'reward': 3},   # 5단계: 4x4, 8페어 (트릭카드)
    {'pairs': 10, 'time_limit': 21.0, 'reward': 3},  # 6단계: 4x5, 10페어
    {'pairs': 10, 'time_limit': 25.0, 'reward': 6}   # 7단계: 4x5, 10페어 (트릭카드)
]


In [ ]:
import numpy as np
from typing import Dict, List, Tuple



def calculate_stage_completion_time(n_pairs: int, skill_level: float) -> float:
    """특정 단계의 예상 완료 시간 계산"""
    baseline_matrix = get_optimal_baseline_markov_chain(n_pairs, entire_matrix=True)
    baseline_stats = get_case_statistics(baseline_matrix, n_pairs)
    baseline_expected_failures = sum(i * prob for i, prob in enumerate(baseline_stats))
    baseline_turns = baseline_expected_failures + n_pairs
    
    # 실력에 따른 실제 턴 수
    actual_turns = calculate_turn_penalty(skill_level, baseline_turns)
    
    # 시간 계산 (턴 * 0.5초)
    completion_time = actual_turns * TURN_DURATION
    
    return completion_time

def can_complete_stage_in_time(n_pairs: int, skill_level: float, time_limit: float) -> Tuple[bool, float]:
    """주어진 시간 내에 단계를 완료할 수 있는지 확인"""
    completion_time = calculate_stage_completion_time(n_pairs, skill_level)
    can_complete = completion_time <= time_limit
    return can_complete, completion_time

def simulate_single_game_session(skill_level: float, verbose: bool = False) -> Dict:
    """단일 게임 세션 시뮬레이션"""
    session_result = {
        'skill_level': skill_level,
        'completed_stages': 0,
        'final_reward': 0,
        'total_ads_watched': 0,
        'total_ad_revenue': 0,
        'session_attempts': 0,
        'stage_details': []
    }
    
    for attempt in range(MAX_RETRIES + 1):  # 최초 시도 + 3회 재시도
        session_result['session_attempts'] += 1
        current_attempt = {
            'attempt_number': attempt + 1,
            'stages_completed': 0,
            'failed_at_stage': None,
            'stage_results': []
        }
        
        # 각 단계별 진행
        for stage_idx, stage in enumerate(GAME_STAGES):
            stage_num = stage_idx + 1
            n_pairs = stage['pairs']
            time_limit = stage['time_limit']
            reward = stage['reward']
            
            can_complete, completion_time = can_complete_stage_in_time(n_pairs, skill_level, time_limit)
            
            stage_result = {
                'stage': stage_num,
                'pairs': n_pairs,
                'time_limit': time_limit,
                'completion_time': completion_time,
                'success': can_complete,
                'reward': reward if can_complete else 0
            }
            
            current_attempt['stage_results'].append(stage_result)
            
            if can_complete:
                current_attempt['stages_completed'] += 1
                if verbose:
                    print(f"  단계 {stage_num}: 성공 ({completion_time:.1f}초/{time_limit}초)")
            else:
                current_attempt['failed_at_stage'] = stage_num
                if verbose:
                    print(f"  단계 {stage_num}: 실패 ({completion_time:.1f}초/{time_limit}초)")
                break
        
        # 세션 결과 업데이트
        if current_attempt['failed_at_stage'] is None:
            # 모든 단계 완료
            session_result['completed_stages'] = len(GAME_STAGES)
            session_result['final_reward'] = GAME_STAGES[-1]['reward']  # 최종 단계 보상
            session_result['stage_details'].append(current_attempt)
            break
        else:
            # 실패 - 광고 시청
            if attempt < MAX_RETRIES:  # 재시도 가능
                session_result['total_ads_watched'] += 1
                session_result['total_ad_revenue'] += AD_REVENUE
                if verbose:
                    print(f"  광고 시청 후 재시도 ({attempt + 1}/{MAX_RETRIES})")
            else:
                # 재시도 횟수 초과
                session_result['completed_stages'] = current_attempt['stages_completed']
                if current_attempt['stages_completed'] > 0:
                    # 마지막으로 완료한 단계의 보상
                    last_completed_stage = current_attempt['stages_completed'] - 1
                    session_result['final_reward'] = GAME_STAGES[last_completed_stage]['reward']
                session_result['total_ads_watched'] += 1
                session_result['total_ad_revenue'] += AD_REVENUE
            
            session_result['stage_details'].append(current_attempt)
    
    # 순이익 계산
    session_result['net_profit'] = session_result['total_ad_revenue'] - session_result['final_reward']
    
    return session_result

def analyze_stage_completion_rates(skill_levels: List[float]) -> Dict:
    """실력별 단계 통과율 분석 - 순차적 진행 고려"""
    
    print("=== 실력별 단계 통과율 분석 ===")
    print("실력\\단계", end="")
    for i in range(len(GAME_STAGES)):
        print(f" | {i+1:^6}", end="")
    print(" | 전체")
    print("-" * (10 + 8 * len(GAME_STAGES) + 8))
    
    stage_completion_rates = {}
    
    for skill in skill_levels:
        stage_completion_rates[skill] = []
        print(f"{skill:^8}", end="")
        
        # 순차적으로 각 단계 확인
        can_proceed = True
        overall_completion = 1.0
        
        for stage_idx, stage in enumerate(GAME_STAGES):
            if can_proceed:
                can_complete, completion_time = can_complete_stage_in_time(
                    stage['pairs'], skill, stage['time_limit']
                )
                
                if can_complete:
                    completion_rate = 1.0
                    stage_completion_rates[skill].append(completion_rate)
                else:
                    # 이 단계에서 실패하면 이후 모든 단계 접근 불가
                    completion_rate = 0.0
                    stage_completion_rates[skill].append(completion_rate)
                    can_proceed = False  # 더 이상 진행 불가
                    overall_completion = 0.0
            else:
                # 이전 단계에서 실패했으므로 이 단계에 도달할 수 없음
                completion_rate = 0.0
                stage_completion_rates[skill].append(completion_rate)
            
            print(f" | {completion_rate:^6.0%}", end="")
        
        # 전체 완주율은 모든 단계를 통과한 경우만
        if can_proceed:
            overall_completion = 1.0
        else:
            overall_completion = 0.0
            
        print(f" | {overall_completion:^6.1%}")
    
    print()
    return stage_completion_rates

def enhanced_monte_carlo_simulation(user_skill_distribution: List[float], 
                                  skill_levels: List[float] = [0.1, 0.3, 0.5, 0.7, 0.9],
                                  num_simulations: int = 10000,
                                  verbose_sample: int = 0) -> Dict:
    """개선된 몬테카를로 시뮬레이션"""
    
    print("=== 게임 단계별 설정 ===")
    for i, stage in enumerate(GAME_STAGES):
        print(f"단계 {i+1}: {stage['pairs']}페어, {stage['time_limit']}초, 보상 {stage['reward']}원")
    print()
    
    # 수정된 단계별 통과율 분석
    stage_completion_rates = analyze_stage_completion_rates(skill_levels)
    
    # 나머지 시뮬레이션 로직은 동일...
    all_results = []
    skill_stats = {skill: [] for skill in skill_levels}
    
    for sim in range(num_simulations):
        user_skill = np.random.choice(skill_levels, p=user_skill_distribution)
        
        verbose = sim < verbose_sample
        if verbose:
            print(f"\n=== 시뮬레이션 {sim+1} (실력: {user_skill}) ===")
        
        session_result = simulate_single_game_session(user_skill, verbose)
        all_results.append(session_result)
        skill_stats[user_skill].append(session_result)
    
    # 결과 분석 (이전과 동일)
    total_ad_revenue = sum(r['total_ad_revenue'] for r in all_results)
    total_rewards_paid = sum(r['final_reward'] for r in all_results)
    total_net_profit = total_ad_revenue - total_rewards_paid
    
    avg_ads_per_session = np.mean([r['total_ads_watched'] for r in all_results])
    avg_completed_stages = np.mean([r['completed_stages'] for r in all_results])
    avg_session_attempts = np.mean([r['session_attempts'] for r in all_results])
    
    skill_analysis = {}
    for skill in skill_levels:
        if skill_stats[skill]:
            skill_results = skill_stats[skill]
            skill_analysis[skill] = {
                'count': len(skill_results),
                'avg_completed_stages': np.mean([r['completed_stages'] for r in skill_results]),
                'avg_ads_watched': np.mean([r['total_ads_watched'] for r in skill_results]),
                'avg_reward': np.mean([r['final_reward'] for r in skill_results]),
                'avg_net_profit': np.mean([r['net_profit'] for r in skill_results]),
                'completion_rate': np.mean([1 if r['completed_stages'] == len(GAME_STAGES) else 0 for r in skill_results])
            }
    
    return {
        'simulation_results': all_results,
        'total_simulations': num_simulations,
        'total_ad_revenue': total_ad_revenue,
        'total_rewards_paid': total_rewards_paid,
        'total_net_profit': total_net_profit,
        'avg_ads_per_session': avg_ads_per_session,
        'avg_completed_stages': avg_completed_stages,
        'avg_session_attempts': avg_session_attempts,
        'skill_analysis': skill_analysis,
        'stage_completion_rates': stage_completion_rates
    }


In [ ]:
# 

In [ ]:
# 

In [ ]:
# import numpy as np
# from typing import Dict, List, Tuple

# # 게임 룰 (확정적)
# GAME_STAGES = [
#     {'pairs': 2, 'time_limit': 5.0, 'reward': 2},
#     {'pairs': 3, 'time_limit': 7.0, 'reward': 2},
#     {'pairs': 6, 'time_limit': 9.0, 'reward': 2},
#     {'pairs': 8, 'time_limit': 13.0, 'reward': 2},
#     {'pairs': 8, 'time_limit': 15.0, 'reward': 3},
#     {'pairs': 10, 'time_limit': 21.0, 'reward': 3},
#     {'pairs': 10, 'time_limit': 25.0, 'reward': 6}
# ]

# MAX_RETRIES = 3
# AD_REVENUE = 1.0

# def calculate_stage_completion_time_v6(n_pairs: int, skill_level: float) -> float:
#     """v6의 기존 함수들만 사용하여 완료 시간 계산"""
    
#     # 베이스라인 계산 (v6의 get_optimal_baseline_markov_chain 사용)
#     baseline_matrix = get_optimal_baseline_markov_chain(n_pairs, entire_matrix=True)
#     baseline_stats = get_case_statistics(baseline_matrix, n_pairs)
#     baseline_expected_failures = sum(i * prob for i, prob in enumerate(baseline_stats))
#     baseline_turns = baseline_expected_failures + n_pairs
    
#     # 더 가파른 패널티 곡선
#     if skill_level >= 0.9:
#         penalty_multiplier = 1.0
#     elif skill_level >= 0.8:
#         penalty_multiplier = 1.2    # 20% 증가
#     elif skill_level >= 0.7:
#         penalty_multiplier = 1.5    # 50% 증가  
#     elif skill_level >= 0.6:
#         penalty_multiplier = 2.0    # 100% 증가
#     elif skill_level >= 0.5:
#         penalty_multiplier = 2.8    # 180% 증가
#     elif skill_level >= 0.4:
#         penalty_multiplier = 4.0    # 300% 증가
#     elif skill_level >= 0.3:
#         penalty_multiplier = 6.0    # 500% 증가
#     else:
#         penalty_multiplier = 10.0   # 900% 증가
    
#     actual_turns = baseline_turns * penalty_multiplier
#     return actual_turns * TURN_DURATION

# def can_complete_stage_v6(n_pairs: int, skill_level: float, time_limit: float) -> Tuple[bool, float]:
#     """v6 함수들만 사용한 완료 가능성 판단"""
    
#     # 1. 베이스라인 대비 완료율 체크
#     baseline_matrix = get_optimal_baseline_markov_chain(n_pairs, entire_matrix=True)
#     baseline_stats = get_case_statistics(baseline_matrix, n_pairs)
#     baseline_completion_rate = sum(baseline_stats)
    
#     # 실력이 매우 낮으면 완료 불가능으로 판단
#     if skill_level < 0.2 and n_pairs >= 6:
#         return False, float('inf')
#     elif skill_level < 0.4 and n_pairs >= 8:
#         return False, float('inf')
    
#     # 2. 시간 계산
#     completion_time = calculate_stage_completion_time_v6(n_pairs, skill_level)
    
#     # 3. 시간 제한 체크
#     can_finish_in_time = completion_time <= time_limit
    
#     return can_finish_in_time, completion_time

# def simulate_session_v6_only(skill_level: float) -> Dict:
#     """v6 함수들만 사용한 세션 시뮬레이션"""
    
#     session_result = {
#         'skill_level': skill_level,
#         'completed_stages': 0,
#         'final_reward': 0,
#         'total_ads_watched': 0,
#         'total_ad_revenue': 0.0,
#         'total_session_time': 0.0,
#         'attempts': [],
#         'net_profit': 0.0
#     }
    
#     for attempt in range(MAX_RETRIES + 1):
#         attempt_result = {
#             'attempt_number': attempt + 1,
#             'stages_completed': 0,
#             'failed_at_stage': None,
#             'stage_times': [],
#             'total_time': 0.0
#         }
        
#         # 순차적으로 모든 단계 시도
#         for stage_num, stage in enumerate(GAME_STAGES, 1):
#             can_complete, completion_time = can_complete_stage_v6(
#                 stage['pairs'], skill_level, stage['time_limit']
#             )
            
#             attempt_result['stage_times'].append({
#                 'stage': stage_num,
#                 'completion_time': completion_time,
#                 'time_limit': stage['time_limit'],
#                 'success': can_complete
#             })
            
#             attempt_result['total_time'] += min(completion_time, stage['time_limit'] * 1.5)
            
#             if can_complete:
#                 attempt_result['stages_completed'] += 1
#             else:
#                 attempt_result['failed_at_stage'] = stage_num
#                 break  # 실패 시 즉시 중단
        
#         session_result['attempts'].append(attempt_result)
#         session_result['total_session_time'] += attempt_result['total_time']
        
#         # 성공/실패 처리
#         if attempt_result['failed_at_stage'] is None:
#             # 모든 단계 완료
#             session_result['completed_stages'] = len(GAME_STAGES)
#             session_result['final_reward'] = GAME_STAGES[-1]['reward']
#             break
#         else:
#             # 실패 처리
#             session_result['completed_stages'] = max(
#                 session_result['completed_stages'], 
#                 attempt_result['stages_completed']
#             )
            
#             if attempt < MAX_RETRIES:
#                 # 재시도 - 광고 시청
#                 session_result['total_ads_watched'] += 1
#                 session_result['total_ad_revenue'] += AD_REVENUE
#             else:
#                 # 최종 실패
#                 session_result['total_ads_watched'] += 1
#                 session_result['total_ad_revenue'] += AD_REVENUE
                
#                 # 마지막 완료 단계 보상
#                 if session_result['completed_stages'] > 0:
#                     last_stage_idx = session_result['completed_stages'] - 1
#                     session_result['final_reward'] = GAME_STAGES[last_stage_idx]['reward']
#                 break
    
#     session_result['net_profit'] = session_result['total_ad_revenue'] - session_result['final_reward']
#     return session_result

# def sample_skill_continuous(cluster_means: List[float], 
#                            cluster_weights: List[float], 
#                            std_dev: float = 0.05) -> float:
#     """연속 실력 분포 샘플링"""
    
#     # 가중치 정규화
#     cluster_weights = np.array(cluster_weights)
#     cluster_weights = cluster_weights / cluster_weights.sum()
    
#     # 클러스터 선택
#     selected_mean = np.random.choice(cluster_means, p=cluster_weights)
    
#     # 정규분포 샘플링
#     sampled_skill = np.random.normal(selected_mean, std_dev)
    
#     # 범위 제한
#     return np.clip(sampled_skill, 0.01, 0.99)

# def run_v6_monte_carlo(cluster_means: List[float],
#                       cluster_weights: List[float],
#                       num_simulations: int = 10000,
#                       skill_std_dev: float = 0.05) -> Dict:
#     """v6 함수들만 사용한 몬테카를로 시뮬레이션"""
    
#     # 입력 검증
#     assert len(cluster_means) == len(cluster_weights), "평균과 가중치 개수 불일치"
#     assert abs(sum(cluster_weights) - 1.0) < 1e-6, "가중치 합이 1이 아님"
    
#     results = {
#         'params': {
#             'cluster_means': cluster_means,
#             'cluster_weights': cluster_weights,
#             'num_simulations': num_simulations,
#             'skill_std_dev': skill_std_dev
#         },
#         'stage_stats': {
#             'attempts': [0] * len(GAME_STAGES),
#             'failures': [0] * len(GAME_STAGES),
#             'failure_rates': [0.0] * len(GAME_STAGES)
#         },
#         'financial': {
#             'total_ad_revenue': 0.0,
#             'total_rewards': 0.0,
#             'net_profit': 0.0
#         },
#         'session_stats': {
#             'avg_time': 0.0,
#             'avg_ads': 0.0,
#             'avg_stages': 0.0,
#             'stage_distribution': [0] * (len(GAME_STAGES) + 1)
#         },
#         'skill_analysis': {}
#     }
    
#     # 시뮬레이션 실행
#     for sim in range(num_simulations):
#         # 실력 샘플링
#         skill = sample_skill_continuous(cluster_means, cluster_weights, skill_std_dev)
        
#         # 세션 시뮬레이션
#         session = simulate_session_v6_only(skill)
        
#         # 데이터 집계
#         results['financial']['total_ad_revenue'] += session['total_ad_revenue']
#         results['financial']['total_rewards'] += session['final_reward']
#         results['session_stats']['avg_time'] += session['total_session_time']
#         results['session_stats']['avg_ads'] += session['total_ads_watched']
#         results['session_stats']['avg_stages'] += session['completed_stages']
        
#         # 완료 단계 분포
#         results['session_stats']['stage_distribution'][session['completed_stages']] += 1
        
#         # 단계별 실패 집계
#         for attempt in session['attempts']:
#             for stage_idx in range(len(GAME_STAGES)):
#                 if stage_idx < len(attempt['stage_times']):
#                     results['stage_stats']['attempts'][stage_idx] += 1
#                     if not attempt['stage_times'][stage_idx]['success']:
#                         results['stage_stats']['failures'][stage_idx] += 1
        
#         # 실력별 분석
#         skill_bin = round(skill * 10) / 10
#         if skill_bin not in results['skill_analysis']:
#             results['skill_analysis'][skill_bin] = {
#                 'count': 0, 'stages': 0, 'profit': 0, 'ads': 0
#             }
        
#         data = results['skill_analysis'][skill_bin]
#         data['count'] += 1
#         data['stages'] += session['completed_stages']
#         data['profit'] += session['net_profit']
#         data['ads'] += session['total_ads_watched']
    
#     # 평균 계산
#     results['financial']['net_profit'] = (
#         results['financial']['total_ad_revenue'] - results['financial']['total_rewards']
#     )
#     results['session_stats']['avg_time'] /= num_simulations
#     results['session_stats']['avg_ads'] /= num_simulations
#     results['session_stats']['avg_stages'] /= num_simulations
    
#     # 실패율 계산
#     for i in range(len(GAME_STAGES)):
#         attempts = results['stage_stats']['attempts'][i]
#         if attempts > 0:
#             results['stage_stats']['failure_rates'][i] = results['stage_stats']['failures'][i] / attempts
    
#     # 실력별 평균
#     for skill_bin, data in results['skill_analysis'].items():
#         if data['count'] > 0:
#             data['avg_stages'] = data['stages'] / data['count']
#             data['avg_profit'] = data['profit'] / data['count']
#             data['avg_ads'] = data['ads'] / data['count']
    
#     return results

# def print_v6_results(results: Dict):
#     """결과 출력"""
#     print("=== V6 몬테카를로 시뮬레이션 결과 ===")
#     print(f"시뮬레이션: {results['params']['num_simulations']:,}회")
#     print(f"실력 클러스터: {list(zip(results['params']['cluster_means'], results['params']['cluster_weights']))}")
#     print()
    
#     print("=== 재무 성과 ===")
#     financial = results['financial']
#     print(f"광고 수익: {financial['total_ad_revenue']:,.0f}원")
#     print(f"보상 지급: {financial['total_rewards']:,.0f}원") 
#     print(f"순이익: {financial['net_profit']:,.0f}원")
#     print()
    
#     print("=== 세션 통계 ===")
#     session = results['session_stats']
#     print(f"평균 시간: {session['avg_time']:.1f}초")
#     print(f"평균 광고: {session['avg_ads']:.2f}회")
#     print(f"평균 단계: {session['avg_stages']:.2f}")
#     print()
    
#     print("=== 단계별 실패율 ===")
#     for i, rate in enumerate(results['stage_stats']['failure_rates']):
#         stage = GAME_STAGES[i]
#         print(f"단계{i+1} ({stage['pairs']}페어, {stage['time_limit']}초): {rate:.1%}")
#     print()
    
#     print("=== 실력별 성과 ===")
#     for skill in sorted(results['skill_analysis'].keys()):
#         data = results['skill_analysis'][skill]
#         print(f"실력{skill:.1f}: {data['avg_stages']:.1f}단계, {data['avg_ads']:.1f}광고, "
#               f"{data['avg_profit']:.1f}원 (샘플:{data['count']})")

# # 실행 함수
# def execute_v6_simulation():
#     """v6 시뮬레이션 실행"""
    
#     user_clusters = {
#         'means': [0.15, 0.25, 0.35, 0.45, 0.55, 0.70, 0.9],
#         'weights': [0.05, 0.10, 0.25, 0.30, 0.20, 0.09, 0.01]
#     }
    
#     results = run_v6_monte_carlo(
#         cluster_means=user_clusters['means'],
#         cluster_weights=user_clusters['weights'],
#         num_simulations=10000,
#         skill_std_dev=0.06
#     )
    
#     print_v6_results(results)

In [ ]:
execute_v6_simulation()

=== V6 몬테카를로 시뮬레이션 결과 ===
시뮬레이션: 10,000회
실력 클러스터: [(0.15, 0.05), (0.25, 0.1), (0.35, 0.25), (0.45, 0.3), (0.55, 0.2), (0.7, 0.09), (0.9, 0.01)]

=== 재무 성과 ===
광고 수익: 37,756원
보상 지급: 8,792원
순이익: 28,964원

=== 세션 통계 ===
평균 시간: 43.1초
평균 광고: 3.78회
평균 단계: 0.94

=== 단계별 실패율 ===
단계1 (2페어, 5.0초): 70.2%
단계2 (3페어, 7.0초): 0.0%
단계3 (6페어, 9.0초): 95.1%
단계4 (8페어, 13.0초): 0.0%
단계5 (8페어, 15.0초): 0.0%
단계6 (10페어, 21.0초): 0.0%
단계7 (10페어, 25.0초): 0.0%

=== 실력별 성과 ===
실력0.0: 0.0단계, 4.0광고, 4.0원 (샘플:28)
실력0.1: 0.0단계, 4.0광고, 4.0원 (샘플:302)
실력0.2: 0.0단계, 4.0광고, 4.0원 (샘플:782)
실력0.3: 0.0단계, 4.0광고, 4.0원 (샘플:1766)
실력0.4: 0.0단계, 4.0광고, 4.0원 (샘플:2631)
실력0.5: 1.0단계, 4.0광고, 3.0원 (샘플:2341)
실력0.6: 2.0단계, 4.0광고, 2.0원 (샘플:1207)
실력0.7: 4.2단계, 2.3광고, -1.5원 (샘플:676)
실력0.8: 7.0단계, 0.0광고, -6.0원 (샘플:187)
실력0.9: 7.0단계, 0.0광고, -6.0원 (샘플:63)
실력1.0: 7.0단계, 0.0광고, -6.0원 (샘플:17)


In [ ]:
# 

In [ ]:
# 

### 로지스틱 변량

In [ ]:
# import numpy as np
# from typing import Dict, List, Tuple

# # 게임 룰 (동일)
# GAME_STAGES = [
#     {'pairs': 2, 'time_limit': 5.0, 'reward': 2},
#     {'pairs': 3, 'time_limit': 7.0, 'reward': 2},
#     {'pairs': 6, 'time_limit': 9.0, 'reward': 2},
#     {'pairs': 8, 'time_limit': 13.0, 'reward': 2},
#     {'pairs': 8, 'time_limit': 15.0, 'reward': 3},
#     {'pairs': 10, 'time_limit': 21.0, 'reward': 3},
#     {'pairs': 10, 'time_limit': 25.0, 'reward': 6}
# ]

# MAX_RETRIES = 3
# AD_REVENUE = 1.0

# def logistic_penalty_curve(skill_level: float, steepness: float = 8.0, midpoint: float = 0.5) -> float:
#     """로지스틱 곡선 기반 연속적 페널티 계산
    
#     Args:
#         skill_level: 0.01~0.99 실력 수준
#         steepness: 곡선의 가파른 정도 (높을수록 가파름)
#         midpoint: 변곡점 (0.5 = 중간 실력에서 급변)
    
#     Returns:
#         penalty_multiplier: 1.0 (베이스라인) ~ 10.0 (최대 페널티)
#     """
    
#     # 로지스틱 함수: 1 / (1 + e^(-k*(x-x0)))
#     # 역로지스틱으로 높은 실력 = 낮은 페널티
#     x = skill_level
#     k = steepness
#     x0 = midpoint
    
#     # 0~1 범위의 로지스틱 값
#     logistic_value = 1 / (1 + np.exp(-k * (x - x0)))
    
#     # 페널티 범위: 10.0 (최악) ~ 1.0 (최고)
#     min_penalty = 1.0
#     max_penalty = 10.0
    
#     # 역으로 매핑: 높은 실력 → 낮은 페널티
#     penalty_multiplier = max_penalty - (max_penalty - min_penalty) * logistic_value
    
#     return penalty_multiplier

# def calculate_continuous_completion_time(n_pairs: int, skill_level: float) -> float:
#     """연속적 페널티 곡선을 사용한 완료 시간 계산"""
    
#     # 베이스라인 계산
#     baseline_matrix = get_optimal_baseline_markov_chain(n_pairs, entire_matrix=True)
#     baseline_stats = get_case_statistics(baseline_matrix, n_pairs)
#     baseline_expected_failures = sum(i * prob for i, prob in enumerate(baseline_stats))
#     baseline_turns = baseline_expected_failures + n_pairs
    
#     # 로지스틱 곡선 기반 연속적 페널티
#     penalty_multiplier = logistic_penalty_curve(skill_level)
    
#     actual_turns = baseline_turns * penalty_multiplier
#     return actual_turns * TURN_DURATION
# def calculate_success_probability(completion_time: float, time_limit: float) -> float:
#     """완료 시간과 제한 시간을 바탕으로 성공 확률 계산"""
    
#     time_ratio = completion_time / time_limit
    
#     if time_ratio <= 0.8:       # 여유로운 경우
#         return 0.95
#     elif time_ratio <= 1.0:     # 아슬아슬한 경우  
#         # 선형 감소: 0.8→1.0 구간에서 0.95→0.7
#         slope = (0.7 - 0.95) / (1.0 - 0.8)
#         return 0.95 + slope * (time_ratio - 0.8)
#     elif time_ratio <= 1.2:     # 시간 초과하지만 가능성 있음
#         # 선형 감소: 1.0→1.2 구간에서 0.7→0.1  
#         slope = (0.1 - 0.7) / (1.2 - 1.0)
#         return 0.7 + slope * (time_ratio - 1.0)
#     else:                       # 완전히 불가능
#         return 0.05
    
# def can_complete_stage_continuous(n_pairs: int, skill_level: float, time_limit: float) -> Tuple[bool, float]:
#     """연속적 페널티를 사용한 완료 가능성 판단"""
    
#     # 완료 시간 계산 (연속적)
#     completion_time = calculate_continuous_completion_time(n_pairs, skill_level)
    
#     # 시간 제한 체크 (순수하게 시간만으로 판단)
#     can_finish_in_time = completion_time <= time_limit
    
#     return can_finish_in_time, completion_time
# def can_complete_stage_probabilistic(n_pairs: int, skill_level: float, time_limit: float) -> Tuple[bool, float]:
#     """확률적 성공/실패 판단"""
    
#     # 1. 기대 완료 시간 계산
#     expected_completion_time = calculate_continuous_completion_time(n_pairs, skill_level)
    
#     # 2. 성공 확률 계산
#     success_probability = calculate_success_probability(expected_completion_time, time_limit)
    
#     # 3. 확률적 성공/실패 결정
#     is_success = np.random.random() < success_probability
    
#     # 4. 실제 소요 시간 (변량 추가)
#     if is_success:
#         # 성공 시: 기대 시간 ± 15% 변량
#         actual_time = expected_completion_time * np.random.uniform(0.85, 1.15)
#         actual_time = min(actual_time, time_limit)  # 시간 제한 내에서
#     else:
#         # 실패 시: 시간 제한 근처에서 포기
#         actual_time = time_limit * np.random.uniform(0.95, 1.3)
    
#     return is_success, actual_time

# def simulate_continuous_session(skill_level: float) -> Dict:
#     """연속적 페널티를 사용한 세션 시뮬레이션"""
    
#     session_result = {
#         'skill_level': skill_level,
#         'completed_stages': 0,
#         'final_reward': 0,
#         'total_ads_watched': 0,
#         'total_ad_revenue': 0.0,
#         'total_session_time': 0.0,
#         'attempts': [],
#         'net_profit': 0.0
#     }
    
#     for attempt in range(MAX_RETRIES + 1):
#         attempt_result = {
#             'attempt_number': attempt + 1,
#             'stages_completed': 0,
#             'failed_at_stage': None,
#             'stage_times': [],
#             'total_time': 0.0
#         }
        
#         # 순차적 단계 진행
#         for stage_num, stage in enumerate(GAME_STAGES, 1):
#             # 확률적 성공/실패 판정
#             is_success, actual_time = can_complete_stage_probabilistic(
#                 stage['pairs'], skill_level, stage['time_limit']
#             )
            
#             attempt_result['stage_times'].append({
#                 'stage': stage_num,
#                 'completion_time': actual_time,
#                 'time_limit': stage['time_limit'],
#                 'success': is_success
#             })
            
#             attempt_result['total_time'] += actual_time
            
#             if is_success:
#                 attempt_result['stages_completed'] += 1
#             else:
#                 attempt_result['failed_at_stage'] = stage_num
#                 break
        
#         session_result['attempts'].append(attempt_result)
#         session_result['total_session_time'] += attempt_result['total_time']
        
        
#         # 성공/실패 처리
#         if attempt_result['failed_at_stage'] is None:
#             # 모든 단계 완료
#             session_result['completed_stages'] = len(GAME_STAGES)
#             session_result['final_reward'] = GAME_STAGES[-1]['reward']
#             break
#         else:
#             # 실패 처리
#             session_result['completed_stages'] = max(
#                 session_result['completed_stages'], 
#                 attempt_result['stages_completed']
#             )
            
#             if attempt < MAX_RETRIES:
#                 session_result['total_ads_watched'] += 1
#                 session_result['total_ad_revenue'] += AD_REVENUE
#             else:
#                 session_result['total_ads_watched'] += 1
#                 session_result['total_ad_revenue'] += AD_REVENUE
                
#                 if session_result['completed_stages'] > 0:
#                     last_stage_idx = session_result['completed_stages'] - 1
#                     session_result['final_reward'] = GAME_STAGES[last_stage_idx]['reward']
#                 break
    
#     session_result['net_profit'] = session_result['total_ad_revenue'] - session_result['final_reward']
#     return session_result

# def run_continuous_monte_carlo(cluster_means: List[float],
#                               cluster_weights: List[float],
#                               num_simulations: int = 10000,
#                               skill_std_dev: float = 0.08) -> Dict:
#     """연속적 페널티를 사용한 몬테카를로 시뮬레이션"""
    
#     results = {
#         'params': {
#             'cluster_means': cluster_means,
#             'cluster_weights': cluster_weights,
#             'num_simulations': num_simulations,
#             'skill_std_dev': skill_std_dev
#         },
#         'stage_stats': {
#             'attempts': [0] * len(GAME_STAGES),
#             'failures': [0] * len(GAME_STAGES),
#             'failure_rates': [0.0] * len(GAME_STAGES)
#         },
#         'financial': {
#             'total_ad_revenue': 0.0,
#             'total_rewards': 0.0,
#             'net_profit': 0.0
#         },
#         'session_stats': {
#             'avg_time': 0.0,
#             'avg_ads': 0.0,
#             'avg_stages': 0.0,
#             'stage_distribution': [0] * (len(GAME_STAGES) + 1)
#         },
#         'skill_analysis': {},
#         'penalty_analysis': {}  # 페널티 분석 추가
#     }
    
#     for sim in range(num_simulations):
#         # 연속적 실력 샘플링
#         skill = sample_skill_continuous(cluster_means, cluster_weights, skill_std_dev)
        
#         # 세션 시뮬레이션
#         session = simulate_continuous_session(skill)
        
#         # 데이터 집계
#         results['financial']['total_ad_revenue'] += session['total_ad_revenue']
#         results['financial']['total_rewards'] += session['final_reward']
#         results['session_stats']['avg_time'] += session['total_session_time']
#         results['session_stats']['avg_ads'] += session['total_ads_watched']
#         results['session_stats']['avg_stages'] += session['completed_stages']
        
#         # 완료 단계 분포
#         results['session_stats']['stage_distribution'][session['completed_stages']] += 1
        
#         # 단계별 실패 집계
#         for attempt in session['attempts']:
#             for stage_idx in range(len(GAME_STAGES)):
#                 if stage_idx < len(attempt['stage_times']):
#                     results['stage_stats']['attempts'][stage_idx] += 1
#                     if not attempt['stage_times'][stage_idx]['success']:
#                         results['stage_stats']['failures'][stage_idx] += 1
        
#         # 실력별 분석 (더 세분화: 0.05 단위)
#         skill_bin = round(skill * 20) / 20  # 0.05 단위
#         if skill_bin not in results['skill_analysis']:
#             results['skill_analysis'][skill_bin] = {
#                 'count': 0, 'stages': 0, 'profit': 0, 'ads': 0
#             }
        
#         data = results['skill_analysis'][skill_bin]
#         data['count'] += 1
#         data['stages'] += session['completed_stages']
#         data['profit'] += session['net_profit']
#         data['ads'] += session['total_ads_watched']
        
#         # 페널티 분석 (0.1 단위)
#         penalty = logistic_penalty_curve(skill)
#         penalty_bin = round(skill * 10) / 10
#         if penalty_bin not in results['penalty_analysis']:
#             results['penalty_analysis'][penalty_bin] = {'penalty': penalty, 'count': 1}
#         else:
#             results['penalty_analysis'][penalty_bin]['count'] += 1
    
#     # 평균 계산
#     results['financial']['net_profit'] = (
#         results['financial']['total_ad_revenue'] - results['financial']['total_rewards']
#     )
#     results['session_stats']['avg_time'] /= num_simulations
#     results['session_stats']['avg_ads'] /= num_simulations
#     results['session_stats']['avg_stages'] /= num_simulations
    
#     # 실패율 계산
#     for i in range(len(GAME_STAGES)):
#         attempts = results['stage_stats']['attempts'][i]
#         if attempts > 0:
#             results['stage_stats']['failure_rates'][i] = results['stage_stats']['failures'][i] / attempts
    
#     # 실력별 평균
#     for skill_bin, data in results['skill_analysis'].items():
#         if data['count'] > 0:
#             data['avg_stages'] = data['stages'] / data['count']
#             data['avg_profit'] = data['profit'] / data['count']
#             data['avg_ads'] = data['ads'] / data['count']
    
#     return results

# def sample_skill_continuous(cluster_means: List[float], 
#                            cluster_weights: List[float], 
#                            std_dev: float = 0.08) -> float:
#     """연속 실력 분포 샘플링 (표준편차 증가)"""
    
#     cluster_weights = np.array(cluster_weights)
#     cluster_weights = cluster_weights / cluster_weights.sum()
    
#     selected_mean = np.random.choice(cluster_means, p=cluster_weights)
#     sampled_skill = np.random.normal(selected_mean, std_dev)
    
#     return np.clip(sampled_skill, 0.01, 0.99)

# def print_continuous_results(results: Dict):
#     """연속적 페널티 결과 출력"""
#     print("=== 연속적 페널티 곡선 몬테카를로 결과 ===")
#     print(f"시뮬레이션: {results['params']['num_simulations']:,}회")
#     print(f"실력 표준편차: {results['params']['skill_std_dev']}")
#     print()
    
#     print("=== 페널티 곡선 분석 ===")
#     for skill in sorted(results['penalty_analysis'].keys()):
#         penalty_data = results['penalty_analysis'][skill]
#         penalty = penalty_data['penalty']
#         print(f"실력 {skill:.1f}: 페널티 {penalty:.2f}x (샘플: {penalty_data['count']})")
#     print()
    
#     print("=== 재무 성과 ===")
#     financial = results['financial']
#     print(f"광고 수익: {financial['total_ad_revenue']:,.0f}원")
#     print(f"보상 지급: {financial['total_rewards']:,.0f}원") 
#     print(f"순이익: {financial['net_profit']:,.0f}원")
#     print()
    
#     print("=== 단계별 실패율 (연속적 변화) ===")
#     for i, rate in enumerate(results['stage_stats']['failure_rates']):
#         stage = GAME_STAGES[i]
#         print(f"단계{i+1} ({stage['pairs']}페어, {stage['time_limit']}초): {rate:.1%}")
#     print()
    
#     print("=== 실력별 성과 (세분화) ===")
#     for skill in sorted(results['skill_analysis'].keys()):
#         if results['skill_analysis'][skill]['count'] >= 50:  # 충분한 샘플만
#             data = results['skill_analysis'][skill]
#             print(f"실력{skill:.2f}: {data['avg_stages']:.1f}단계, {data['avg_ads']:.1f}광고, "
#                   f"{data['avg_profit']:.1f}원 (샘플:{data['count']})")

# def execute_continuous_simulation():
#     """연속적 페널티 시뮬레이션 실행"""
    
#     # 더 다양한 실력 분포
#     user_clusters = {
#         'means': [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8],
#         'weights': [0.15, 0.20, 0.25, 0.25, 0.05, 0.07, 0.03]
#     }
    
#     results = run_continuous_monte_carlo(
#         cluster_means=user_clusters['means'],
#         cluster_weights=user_clusters['weights'],
#         num_simulations=50000,
#         skill_std_dev=0.08  # 더 큰 변량
#     )
    
#     print_continuous_results(results)
#     return results

# # 페널티 곡선 시각화를 위한 함수
# def plot_penalty_curve():
#     """페널티 곡선 시각화"""
#     skills = np.linspace(0.01, 0.99, 100)
#     penalties = [logistic_penalty_curve(s) for s in skills]
    
#     print("=== 로지스틱 페널티 곡선 ===")
#     for i in range(0, len(skills), 10):
#         print(f"실력 {skills[i]:.2f}: 페널티 {penalties[i]:.2f}x")

In [ ]:
# 

In [ ]:
# execute_continuous_simulation()

=== 연속적 페널티 곡선 몬테카를로 결과 ===
시뮬레이션: 50,000회
실력 표준편차: 0.08

=== 페널티 곡선 분석 ===
실력 0.0: 페널티 9.78x (샘플: 245)
실력 0.1: 페널티 9.50x (샘플: 2068)
실력 0.2: 페널티 9.02x (샘플: 6189)
실력 0.3: 페널티 8.82x (샘플: 9859)
실력 0.4: 페널티 6.86x (샘플: 11463)
실력 0.5: 페널티 5.64x (샘플: 9739)
실력 0.6: 페널티 3.41x (샘플: 5353)
실력 0.7: 페널티 2.14x (샘플: 3015)
실력 0.8: 페널티 2.00x (샘플: 1601)
실력 0.9: 페널티 1.37x (샘플: 422)
실력 1.0: 페널티 1.24x (샘플: 46)

=== 재무 성과 ===
광고 수익: 192,508원
보상 지급: 42,892원
순이익: 149,616원

=== 단계별 실패율 (연속적 변화!) ===
단계1 (2페어, 5.0초): 82.8%
단계2 (3페어, 7.0초): 48.2%
단계3 (6페어, 9.0초): 77.1%
단계4 (8페어, 13.0초): 29.6%
단계5 (8페어, 15.0초): 8.6%
단계6 (10페어, 21.0초): 5.9%
단계7 (10페어, 25.0초): 5.7%

=== 실력별 성과 (세분화) ===
실력0.00: 0.2단계, 4.0광고, 3.7원 (샘플:119)
실력0.05: 0.2단계, 4.0광고, 3.6원 (샘플:375)
실력0.10: 0.2단계, 4.0광고, 3.7원 (샘플:992)
실력0.15: 0.2단계, 4.0광고, 3.7원 (샘플:1975)
실력0.20: 0.2단계, 4.0광고, 3.6원 (샘플:3035)
실력0.25: 0.2단계, 4.0광고, 3.7원 (샘플:4176)
실력0.30: 0.2단계, 4.0광고, 3.6원 (샘플:4968)
실력0.35: 0.2단계, 4.0광고, 3.6원 (샘플:5505)
실력0.40: 0.2단계, 4.0광고, 3.6원 (샘플:5770)
실력0.4

{'params': {'cluster_means': [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8],
  'cluster_weights': [0.15, 0.2, 0.25, 0.25, 0.05, 0.07, 0.03],
  'num_simulations': 50000,
  'skill_std_dev': 0.08},
 'stage_stats': {'attempts': [194777, 33441, 17337, 3969, 2795, 2556, 2406],
  'failures': [161336, 16104, 13368, 1174, 239, 150, 137],
  'failure_rates': [0.8283113509295246,
   0.48156454651475733,
   0.7710676587644921,
   0.29579239103048627,
   0.08550983899821109,
   0.05868544600938967,
   0.056940980881130504]},
 'financial': {'total_ad_revenue': 192508.0,
  'total_rewards': 42892.0,
  'net_profit': 149616.0},
 'session_stats': {'avg_time': 31.743660318884103,
  'avg_ads': 3.85016,
  'avg_stages': 0.7393,
  'stage_distribution': [33135, 9362, 4303, 737, 108, 45, 41, 2269]},
 'skill_analysis': {0.5: {'count': 4904,
   'stages': 1017,
   'profit': 17724.0,
   'ads': 19616,
   'avg_stages': 0.20738172920065254,
   'avg_profit': 3.6141924959216967,
   'avg_ads': 4.0},
  0.55: {'count': 3787,
   'stage

In [18]:
import numpy as np
from typing import Dict, List, Tuple

# 게임 룰 (동일)
GAME_STAGES = [
    {'pairs': 2, 'time_limit': 5.0, 'reward': 2},
    {'pairs': 3, 'time_limit': 7.0, 'reward': 2},
    {'pairs': 6, 'time_limit': 9.0, 'reward': 2},
    {'pairs': 8, 'time_limit': 13.0, 'reward': 2},
    {'pairs': 8, 'time_limit': 15.0, 'reward': 3},
    {'pairs': 10, 'time_limit': 21.0, 'reward': 3},
    {'pairs': 10, 'time_limit': 25.0, 'reward': 6}
]

MAX_RETRIES = 3
AD_REVENUE = 1.0

def calculate_weighted_penalty(skill_level: float, turn_duration: float = 0.5, 
                              weight_std: float = 0.1) -> float:
    """실력에 따른 가중치를 정규분포로 적용한 페널티 계산
    
    Args:
        skill_level: 0.01~0.99 실력 수준
        turn_duration: 1턴당 소요 시간 (초)
        weight_std: 가중치 변동성 (표준편차)
    
    Returns:
        penalty_multiplier: 페널티 배수
    """
    
    # 기본 페널티 곡선 (로지스틱)
    steepness = 8.0
    midpoint = 0.5
    x = skill_level
    k = steepness
    x0 = midpoint
    
    logistic_value = 1 / (1 + np.exp(-k * (x - x0)))
    base_penalty = 5.0 - 4.0 * logistic_value  # 1.0~10.0
    
    # 가중치 정규분포 적용 (같은 실력이라도 변동성 반영)
    weight = np.random.normal(1.0, weight_std)  # 평균 1.0, 표준편차 weight_std
    weight = np.clip(weight, 0.5, 2.0)  # 0.5~2.0 범위로 제한
    
    # 턴 지속시간에 따른 추가 페널티
    time_penalty = 1.0 + (turn_duration - 0.5) * 0.5  # 0.5초 기준으로 조정
    
    final_penalty = base_penalty * weight * time_penalty
    
    return np.clip(final_penalty, 1.0, 15.0)  # 최종 범위 제한

def calculate_completion_time_with_variation(n_pairs: int, skill_level: float, 
                                           turn_duration: float = 0.5,
                                           weight_std: float = 0.1) -> float:
    """변동성을 고려한 완료 시간 계산
    
    Args:
        n_pairs: 매칭해야 할 카드 쌍의 수
        skill_level: 사용자 실력
        turn_duration: 1턴당 소요 시간 (초)
        weight_std: 가중치 변동성
    
    Returns:
        completion_time: 예상 완료 시간 (초)
    """
    
    # 베이스라인 계산
    baseline_matrix = get_optimal_baseline_markov_chain(n_pairs, entire_matrix=True)
    baseline_stats = get_case_statistics(baseline_matrix, n_pairs)
    baseline_expected_failures = sum(i * prob for i, prob in enumerate(baseline_stats))
    baseline_turns = baseline_expected_failures + n_pairs
    
    # 가중치가 적용된 페널티 계산
    penalty_multiplier = calculate_weighted_penalty(skill_level, turn_duration, weight_std)
    
    # 실제 턴 수 계산
    actual_turns = baseline_turns * penalty_multiplier
    
    # 시간으로 변환
    completion_time = actual_turns * turn_duration
    
    return completion_time

def can_complete_stage_deterministic(n_pairs: int, skill_level: float, time_limit: float,
                                   turn_duration: float = 0.5, weight_std: float = 0.1) -> Tuple[bool, float]:
    """결정적 완료 가능성 판단 (확률적 성공 제거)
    
    Args:
        n_pairs: 카드 쌍 수
        skill_level: 사용자 실력
        time_limit: 시간 제한
        turn_duration: 1턴당 소요 시간
        weight_std: 가중치 변동성
    
    Returns:
        (can_complete, completion_time): (완료 가능 여부, 예상 완료 시간)
    """
    
    # 완료 시간 계산 (변동성 포함)
    completion_time = calculate_completion_time_with_variation(
        n_pairs, skill_level, turn_duration, weight_std
    )
    
    # 시간 제한 체크 (무조건적)
    can_complete = completion_time <= time_limit
    
    return can_complete, completion_time

def simulate_deterministic_session(skill_level: float, turn_duration: float = 0.5,
                                 weight_std: float = 0.1) -> Dict:
    """결정적 세션 시뮬레이션 (확률적 성공 제거)
    
    Args:
        skill_level: 사용자 실력
        turn_duration: 1턴당 소요 시간
        weight_std: 가중치 변동성
    
    Returns:
        session_result: 세션 결과 딕셔너리
    """
    
    session_result = {
        'skill_level': skill_level,
        'turn_duration': turn_duration,
        'weight_std': weight_std,
        'completed_stages': 0,
        'final_reward': 0,
        'total_ads_watched': 0,
        'total_ad_revenue': 0.0,
        'total_session_time': 0.0,
        'attempts': [],
        'net_profit': 0.0
    }
    
    for attempt in range(MAX_RETRIES + 1):
        attempt_result = {
            'attempt_number': attempt + 1,
            'stages_completed': 0,
            'failed_at_stage': None,
            'stage_times': [],
            'total_time': 0.0
        }
        
        # 순차적 단계 진행
        for stage_num, stage in enumerate(GAME_STAGES, 1):
            # 결정적 성공/실패 판정
            can_complete, completion_time = can_complete_stage_deterministic(
                stage['pairs'], skill_level, stage['time_limit'], 
                turn_duration, weight_std
            )
            
            attempt_result['stage_times'].append({
                'stage': stage_num,
                'completion_time': completion_time,
                'time_limit': stage['time_limit'],
                'success': can_complete
            })
            
            attempt_result['total_time'] += completion_time
            
            if can_complete:
                attempt_result['stages_completed'] += 1
            else:
                attempt_result['failed_at_stage'] = stage_num
                break  # 실패 시 즉시 중단
        
        session_result['attempts'].append(attempt_result)
        session_result['total_session_time'] += attempt_result['total_time']
        
        # 성공/실패 처리
        if attempt_result['failed_at_stage'] is None:
            # 모든 단계 완료
            session_result['completed_stages'] = len(GAME_STAGES)
            session_result['final_reward'] = GAME_STAGES[-1]['reward']
            break
        else:
            # 실패 처리
            session_result['completed_stages'] = max(
                session_result['completed_stages'], 
                attempt_result['stages_completed']
            )
            
            if attempt < MAX_RETRIES:
                # 재시도 - 광고 시청
                session_result['total_ads_watched'] += 1
                session_result['total_ad_revenue'] += AD_REVENUE
            else:
                # 최종 실패
                session_result['total_ads_watched'] += 1
                session_result['total_ad_revenue'] += AD_REVENUE
                
                # 마지막 완료 단계 보상
                if session_result['completed_stages'] > 0:
                    last_stage_idx = session_result['completed_stages'] - 1
                    session_result['final_reward'] = GAME_STAGES[last_stage_idx]['reward']
                break
    
    session_result['net_profit'] = session_result['total_ad_revenue'] - session_result['final_reward']
    return session_result

def run_deterministic_monte_carlo(cluster_means: List[float],
                                cluster_weights: List[float],
                                turn_duration: float = 0.5,
                                weight_std: float = 0.1,
                                num_simulations: int = 10000,
                                skill_std_dev: float = 0.08) -> Dict:
    """결정적 몬테카를로 시뮬레이션 (확률적 성공 제거)
    
    Args:
        cluster_means: 실력 클러스터 평균값들
        cluster_weights: 클러스터 가중치들
        turn_duration: 1턴당 소요 시간
        weight_std: 가중치 변동성
        num_simulations: 시뮬레이션 횟수
        skill_std_dev: 실력 샘플링 표준편차
    
    Returns:
        results: 시뮬레이션 결과
    """
    
    results = {
        'params': {
            'cluster_means': cluster_means,
            'cluster_weights': cluster_weights,
            'turn_duration': turn_duration,
            'weight_std': weight_std,
            'num_simulations': num_simulations,
            'skill_std_dev': skill_std_dev
        },
        'stage_stats': {
            'attempts': [0] * len(GAME_STAGES),
            'failures': [0] * len(GAME_STAGES),
            'failure_rates': [0.0] * len(GAME_STAGES)
        },
        'financial': {
            'total_ad_revenue': 0.0,
            'total_rewards': 0.0,
            'net_profit': 0.0
        },
        'session_stats': {
            'avg_time': 0.0,
            'avg_ads': 0.0,
            'avg_stages': 0.0,
            'stage_distribution': [0] * (len(GAME_STAGES) + 1)
        },
        'skill_analysis': {},
        'penalty_analysis': {}
    }
    
    for sim in range(num_simulations):
        # 연속적 실력 샘플링
        skill = sample_skill_continuous(cluster_means, cluster_weights, skill_std_dev)
        
        # 세션 시뮬레이션 (결정적)
        session = simulate_deterministic_session(skill, turn_duration, weight_std)
        
        # 데이터 집계
        results['financial']['total_ad_revenue'] += session['total_ad_revenue']
        results['financial']['total_rewards'] += session['final_reward']
        results['session_stats']['avg_time'] += session['total_session_time']
        results['session_stats']['avg_ads'] += session['total_ads_watched']
        results['session_stats']['avg_stages'] += session['completed_stages']
        
        # 완료 단계 분포
        results['session_stats']['stage_distribution'][session['completed_stages']] += 1
        
        # 단계별 실패 집계
        for attempt in session['attempts']:
            for stage_idx in range(len(GAME_STAGES)):
                if stage_idx < len(attempt['stage_times']):
                    results['stage_stats']['attempts'][stage_idx] += 1
                    if not attempt['stage_times'][stage_idx]['success']:
                        results['stage_stats']['failures'][stage_idx] += 1
        
        # 실력별 분석
        skill_bin = round(skill * 20) / 20  # 0.05 단위
        if skill_bin not in results['skill_analysis']:
            results['skill_analysis'][skill_bin] = {
                'count': 0, 'stages': 0, 'profit': 0, 'ads': 0
            }
        
        data = results['skill_analysis'][skill_bin]
        data['count'] += 1
        data['stages'] += session['completed_stages']
        data['profit'] += session['net_profit']
        data['ads'] += session['total_ads_watched']
        
        # 페널티 분석
        penalty = calculate_weighted_penalty(skill, turn_duration, weight_std)
        penalty_bin = round(skill * 10) / 10
        if penalty_bin not in results['penalty_analysis']:
            results['penalty_analysis'][penalty_bin] = {'penalty': penalty, 'count': 1}
        else:
            results['penalty_analysis'][penalty_bin]['count'] += 1
    
    # 평균 계산
    results['financial']['net_profit'] = (
        results['financial']['total_ad_revenue'] - results['financial']['total_rewards']
    )
    results['session_stats']['avg_time'] /= num_simulations
    results['session_stats']['avg_ads'] /= num_simulations
    results['session_stats']['avg_stages'] /= num_simulations
    
    # 실패율 계산
    for i in range(len(GAME_STAGES)):
        attempts = results['stage_stats']['attempts'][i]
        if attempts > 0:
            results['stage_stats']['failure_rates'][i] = results['stage_stats']['failures'][i] / attempts
    
    # 실력별 평균
    for skill_bin, data in results['skill_analysis'].items():
        if data['count'] > 0:
            data['avg_stages'] = data['stages'] / data['count']
            data['avg_profit'] = data['profit'] / data['count']
            data['avg_ads'] = data['ads'] / data['count']
    
    return results

def sample_skill_continuous(cluster_means: List[float], 
                           cluster_weights: List[float], 
                           std_dev: float = 0.08) -> float:
    """연속 실력 분포 샘플링"""
    
    cluster_weights = np.array(cluster_weights)
    cluster_weights = cluster_weights / cluster_weights.sum()
    
    selected_mean = np.random.choice(cluster_means, p=cluster_weights)
    sampled_skill = np.random.normal(selected_mean, std_dev)
    
    return np.clip(sampled_skill, 0.01, 0.99)

def print_deterministic_results(results: Dict):
    """결정적 시뮬레이션 결과 출력"""
    print("=== 결정적 몬테카를로 시뮬레이션 결과 ===")
    print(f"시뮬레이션: {results['params']['num_simulations']:,}회")
    print(f"턴 지속시간: {results['params']['turn_duration']:.1f}초")
    print(f"가중치 변동성: {results['params']['weight_std']:.2f}")
    print()
    
    print("=== 페널티 분석 (가중치 변동성 포함) ===")
    for skill in sorted(results['penalty_analysis'].keys()):
        penalty_data = results['penalty_analysis'][skill]
        penalty = penalty_data['penalty']
        print(f"실력 {skill:.1f}: 페널티 {penalty:.2f}x (샘플: {penalty_data['count']})")
    print()
    
    print("=== 재무 성과 ===")
    financial = results['financial']
    print(f"광고 수익: {financial['total_ad_revenue']:,.0f}원")
    print(f"보상 지급: {financial['total_rewards']:,.0f}원") 
    print(f"순이익: {financial['net_profit']:,.0f}원")
    print()
    
    print("=== 세션 통계 ===")
    session = results['session_stats']
    print(f"평균 시간: {session['avg_time']:.1f}초")
    print(f"평균 광고: {session['avg_ads']:.2f}회")
    print(f"평균 단계: {session['avg_stages']:.2f}")
    print()
    
    print("=== 단계별 상세 통계 ===")
    print("단계 | 페어수 | 시간제한 | 시도횟수 | 성공횟수 | 실패횟수 | 성공전환율 | 실패율")
    print("-----|--------|----------|----------|----------|----------|------------|--------")
    
    for i in range(len(GAME_STAGES)):
        stage = GAME_STAGES[i]
        attempts = results['stage_stats']['attempts'][i]
        failures = results['stage_stats']['failures'][i]
        successes = attempts - failures
        success_rate = (successes / attempts * 100) if attempts > 0 else 0
        failure_rate = (failures / attempts * 100) if attempts > 0 else 0
        
        print(f"{i+1:^4} | {stage['pairs']:^6} | {stage['time_limit']:^8.1f}초 | {attempts:^8} | {successes:^8} | {failures:^8} | {success_rate:^10.1f}% | {failure_rate:^6.1f}%")
    print()
    
    print("=== 실력별 성과 ===")
    for skill in sorted(results['skill_analysis'].keys()):
        if results['skill_analysis'][skill]['count'] >= 50:
            data = results['skill_analysis'][skill]
            print(f"실력{skill:.2f}: {data['avg_stages']:.1f}단계, {data['avg_ads']:.1f}광고, "
                  f"{data['avg_profit']:.1f}원 (샘플:{data['count']})")

def execute_continuous_simulation(turn_duration, weight_std, user_clusters, skill_std_dev, num_simulations):
    """수정된 연속적 시뮬레이션 실행 (확률적 성공 제거, TURN_DURATION 변수화)
    
    Args:
        turn_duration: 1턴당 소요 시간 (초)
        weight_std: 가중치 변동성 (표준편차)
    """
    
    # 사용자 실력 분포
    
    results = run_deterministic_monte_carlo(
        cluster_means=user_clusters['means'],
        cluster_weights=user_clusters['weights'],
        turn_duration=turn_duration,
        weight_std=weight_std,
        num_simulations=num_simulations,
        skill_std_dev=skill_std_dev
    )
    
    print_deterministic_results(results)
    return results



In [19]:
num_simulations = 5000
user_clusters = {
    'means': [0.5, 0.6, 0.7, 0.8],
    'weights': [0.7, 0.1, 0.1, 0.1]
}
skill_std_dev = 0.02

# 기본 설정으로 테스트
print(f"1. 기본 설정 (턴 지속시간: 0.8초, 가중치 변동성: 0.3)")
results1 = execute_continuous_simulation(
    turn_duration=0.8, 
    weight_std=0.3, 
    user_clusters=user_clusters, 
    skill_std_dev=skill_std_dev, 
    num_simulations=num_simulations
)

1. 기본 설정 (턴 지속시간: 0.8초, 가중치 변동성: 0.3)
=== 결정적 몬테카를로 시뮬레이션 결과 ===
시뮬레이션: 5,000회
턴 지속시간: 0.8초
가중치 변동성: 0.30

=== 페널티 분석 (가중치 변동성 포함) ===
실력 0.4: 페널티 5.26x (샘플: 21)
실력 0.5: 페널티 4.87x (샘플: 3450)
실력 0.6: 페널티 2.22x (샘플: 531)
실력 0.7: 페널티 1.94x (샘플: 518)
실력 0.8: 페널티 2.16x (샘플: 478)
실력 0.9: 페널티 1.00x (샘플: 2)

=== 재무 성과 ===
광고 수익: 19,912원
보상 지급: 6,173원
순이익: 13,739원

=== 세션 통계 ===
평균 시간: 46.5초
평균 광고: 3.98회
평균 단계: 0.98

=== 단계별 상세 통계 ===
단계 | 페어수 | 시간제한 | 시도횟수 | 성공횟수 | 실패횟수 | 성공전환율 | 실패율
-----|--------|----------|----------|----------|----------|------------|--------
 1   |   2    |   5.0   초 |  19950   |   6255   |  13695   |    31.4   % |  68.6 %
 2   |   3    |   7.0   초 |   6255   |   2835   |   3420   |    45.3   % |  54.7 %
 3   |   6    |   9.0   초 |   2835   |   489    |   2346   |    17.2   % |  82.8 %
 4   |   8    |   13.0  초 |   489    |   131    |   358    |    26.8   % |  73.2 %
 5   |   8    |   15.0  초 |   131    |    60    |    71    |    45.8   % |  54.2 %
 6   |   10   |   21.0 

In [5]:
print("\n" + "="*60)
print("2. 다른 설정으로 테스트 (턴 지속시간: 0.3초, 가중치 변동성: 0.15)")
results2 = execute_continuous_simulation(turn_duration=0.3, weight_std=0.15)



2. 다른 설정으로 테스트 (턴 지속시간: 0.3초, 가중치 변동성: 0.15)
=== 결정적 몬테카를로 시뮬레이션 결과 ===
시뮬레이션: 50,000회
턴 지속시간: 0.3초
가중치 변동성: 0.15

=== 페널티 분석 (가중치 변동성 포함) ===
실력 0.0: 페널티 7.14x (샘플: 217)
실력 0.1: 페널티 10.53x (샘플: 2182)
실력 0.2: 페널티 6.73x (샘플: 6205)
실력 0.3: 페널티 7.89x (샘플: 9688)
실력 0.4: 페널티 4.71x (샘플: 11540)
실력 0.5: 페널티 4.69x (샘플: 9767)
실력 0.6: 페널티 4.53x (샘플: 5331)
실력 0.7: 페널티 2.63x (샘플: 2876)
실력 0.8: 페널티 1.78x (샘플: 1685)
실력 0.9: 페널티 1.18x (샘플: 465)
실력 1.0: 페널티 1.04x (샘플: 44)

=== 재무 성과 ===
광고 수익: 171,789원
보상 지급: 100,907원
순이익: 70,882원

=== 세션 통계 ===
평균 시간: 47.7초
평균 광고: 3.44회
평균 단계: 1.96

=== 단계별 실패율 (결정적) ===
단계1 (2페어, 5.0초): 53.9%
단계2 (3페어, 7.0초): 40.3%
단계3 (6페어, 9.0초): 78.9%
단계4 (8페어, 13.0초): 24.2%
단계5 (8페어, 15.0초): 3.6%
단계6 (10페어, 21.0초): 0.6%
단계7 (10페어, 25.0초): 0.0%

=== 실력별 성과 ===
실력0.00: 0.2단계, 4.0광고, 3.6원 (샘플:105)
실력0.05: 0.1단계, 4.0광고, 3.8원 (샘플:335)
실력0.10: 0.1단계, 4.0광고, 3.8원 (샘플:1058)
실력0.15: 0.1단계, 4.0광고, 3.7원 (샘플:2066)
실력0.20: 0.2단계, 4.0광고, 3.7원 (샘플:3093)
실력0.25: 0.3단계, 4.0광고, 3.5원 (샘플:4214)
실력0

In [ ]:
# print("\n" + "="*60)
# print("3. 가중치 변동성 비교 테스트")
# print()

# # 가중치 변동성에 따른 페널티 변화 확인
# import numpy as np

# def test_weight_variation():
#     """가중치 변동성 테스트"""
#     skill_level = 0.5
#     turn_duration = 0.5
    
#     print("실력 0.5, 턴 지속시간 0.5초에서 가중치 변동성 테스트:")
#     print()
    
#     for weight_std in [0.05, 0.1, 0.2, 0.3]:
#         penalties = []
#         for _ in range(1000):  # 1000회 샘플링
#             penalty = calculate_weighted_penalty(skill_level, turn_duration, weight_std)
#             penalties.append(penalty)
        
#         penalties = np.array(penalties)
#         print(f"가중치 변동성 {weight_std:.2f}: 평균 {penalties.mean():.2f}, "
#               f"표준편차 {penalties.std():.2f}, 범위 [{penalties.min():.2f}, {penalties.max():.2f}]")

# test_weight_variation()



3. 가중치 변동성 비교 테스트

실력 0.5, 턴 지속시간 0.5초에서 가중치 변동성 테스트:



NameError: name 'calculate_weighted_penalty' is not defined

In [ ]:
# plot_penalty_curve()

=== 로지스틱 페널티 곡선 ===
실력 0.01: 페널티 9.82x
실력 0.11: 페널티 9.62x
실력 0.21: 페널티 9.21x
실력 0.31: 페널티 8.42x
실력 0.41: 페널티 7.12x
실력 0.50: 페널티 5.41x
실력 0.60: 페널티 3.73x
실력 0.70: 페널티 2.48x
실력 0.80: 페널티 1.74x
실력 0.90: 페널티 1.35x
